In [2]:
import os
import pandas as pd
import numpy as np

# we are going to go quarter by quarter and see if there are any large airport routes from major carriers that dissapear from quarter to quarter
# this can be signified by the top carrier in a route disappearing and the amount of people using that route year over year dropping
# let's look into more possibilities if we have time

path_to_routes = "/Users/tristanbrigham/Desktop/Citadel Datathon/Local Important Data/Consumer_Airfare_Report__Table_6_-_Contiguous_State_City-Pair_Markets_That_Average_At_Least_10_Passengers_Per_Day.csv"

routes_df = pd.read_csv(path_to_routes)

custom_id = "citadel_id"


max_passenger_change = 0.15      # the largest percentage change in passengers that we will allow without flagging per quarter
ms_del = 0.25                    # the largest percentage change in ms that we will tolerate without flagging

/var/folders/d2/lsvbp7p92gnc5ql59sc72rx40000gn/T/ipykernel_44548/858749288.py:11: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  routes_df = pd.read_csv(path_to_routes)


In [3]:
# This function is used to create the custom codes that we use
# to index the airport pairing mappings
def conv_str(num1, num2):

    s = sorted([num1, num2])
    return str(s[0]) + str(s[1])


# apply the code generation function
routes_df[custom_id] = routes_df.apply(lambda row: conv_str(row['citymarketid_1'], row['citymarketid_2']), axis=1)
routes_df["dropped_q_over_q"] = False
routes_df["pass_and_lg_carrier_chng"] = False
routes_df["fare_and_lg_carrier_chng"] = False
routes_df["ms_change"] = False

routes_df["pass_chng_pct"] = 0
routes_df["fare_chng_pct"] = 0
routes_df["lg_fare_chng_pct"] = 0
routes_df["lf_fare_chng_pct"] = 0
routes_df["lg_carr_chng"] = 0
routes_df["lf_carr_chng"] = 0
routes_df["lg_ms_chng"] = 0
routes_df["lf_ms_chng"] = 0


In [4]:
cols_frame = routes_df.columns
S_TEST = True
total_cancelled = pd.DataFrame(columns = cols_frame)

# the first, last dataset
last_q = 1996
last_y = 4
last_q_mappings = routes_df[(routes_df["Year"] == last_q) & (routes_df["quarter"] == last_y)]


# go through each year
for year in range(1997, 2023):

    print(f"YEAR: {year}")

    for quarter in range(1, 5):

        print(f"QUARTER: {quarter}")

        curr_frame = routes_df[(routes_df["Year"] == year) & (routes_df["quarter"] == quarter)]


        ## CHECK 1
        # check 1: check if any disappeared quarter over quarter last_q_mappings
        dropped_routes = last_q_mappings[~last_q_mappings[custom_id].isin(curr_frame[custom_id])]
        dropped_routes["dropped_q_over_q"] = True
        total_cancelled = pd.concat([total_cancelled, dropped_routes], ignore_index=True)


        # # CHECK 2:
        # check 2: check if variables below (largest carrier change, amt people change, mkt share change) mean major player left
        # going to start with decrease in passengers
        # we are going to look whether the carrier that is the largest and the carrier that is the smallest ended up changing

        for idx, row in curr_frame.iterrows():

            if idx % 100 is 0:

                print(f"IDX: {idx}")


            # trying incase that flight didn't exist the year before
            try:
    
                pass_pct_chng = row["passengers"] / routes_df[(routes_df["Year"] == last_q) & (routes_df["quarter"] == last_y) & (routes_df[custom_id] == row[custom_id])]["passengers"].values[0]
                fare_pct_chng = row["fare"] / routes_df[(routes_df["Year"] == last_q) & (routes_df["quarter"] == last_y) & (routes_df[custom_id] == row[custom_id])]["fare"].values[0]
                fare_lg_pct_chng = row["fare_lg"] / routes_df[(routes_df["Year"] == last_q) & (routes_df["quarter"] == last_y) & (routes_df[custom_id] == row[custom_id])]["fare_lg"].values[0]
                fare_low_pct_chng = row["fare_low"] / routes_df[(routes_df["Year"] == last_q) & (routes_df["quarter"] == last_y) & (routes_df[custom_id] == row[custom_id])]["fare_low"].values[0]
                
                F_lg_carr_chng = row["carrier_lg"] != routes_df.loc[(routes_df["Year"] == last_q) & (routes_df["quarter"] == last_y) & (routes_df[custom_id] == row[custom_id])]["carrier_lg"].values[0]
                F_lf_carr_chng = row["carrier_low"] != routes_df.loc[(routes_df["Year"] == last_q) & (routes_df["quarter"] == last_y) & (routes_df[custom_id] == row[custom_id])]["carrier_low"].values[0]
                F_lg_ms_chng = (row["large_ms"] - routes_df[(routes_df["Year"] == last_q) & (routes_df["quarter"] == last_y) & (routes_df[custom_id] == row[custom_id])]["large_ms"].values[0]) 
                F_lf_ms_chng = (row["lf_ms"] - routes_df[(routes_df["Year"] == last_q) & (routes_df["quarter"] == last_y) & (routes_df[custom_id] == row[custom_id])]["lf_ms"].values[0]) 


                # change percentages
                routes_df["pass_chng_pct"] = pass_pct_chng
                routes_df["fare_chng_pct"] = fare_pct_chng
                routes_df["lg_fare_chng_pct"] = fare_lg_pct_chng
                routes_df["lf_fare_chng_pct"] = fare_low_pct_chng
                routes_df["lg_carr_chng"] = F_lg_carr_chng
                routes_df["lf_carr_chng"] = F_lf_carr_chng
                routes_df["lg_ms_chng"] = F_lg_ms_chng
                routes_df["lf_ms_chng"] = F_lf_ms_chng


                # create flags
                F_pass_pct_chng = pass_pct_chng - 1 < max_passenger_change
                F_fare_pct_chng = fare_pct_chng > max_passenger_change
                F_lg_ms_chng = (F_lg_ms_chng - 1) > ms_del
                F_lf_ms_chng = (F_lf_ms_chng - 1) > ms_del

                # if the passenger percentage dropped by too much or fares went up by too much
                # and the top carrier has to change (!)

                if (F_pass_pct_chng and (F_lg_carr_chng or F_ch_carr_chng)):
                    row["pass_and_lg_carrier_chng"] = True
                    total_cancelled = pd.concat([total_cancelled, row], ignore_index=True)
                
                if (F_fare_pct_chng and (F_lg_carr_chng or F_ch_carr_chng)):
                    row["fare_and_lg_carrier_chng"] = True
                    total_cancelled = pd.concat([total_cancelled, row], ignore_index=True)

                if F_lg_ms_chng:
                    row["ms_change"] = True
                    total_cancelled = pd.concat([total_cancelled, row], ignore_index=True)

                    
                if F_lf_ms_chng:
                    row["ms_change"] = True
                    total_cancelled = pd.concat([total_cancelled, row], ignore_index=True)

            
            except:

                pass




        # update the dataframe from last quarter
        last_q_mappings = curr_frame
        last_q = quarter
        last_y = year





<>:37: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:37: SyntaxWarning: "is" with a literal. Did you mean "=="?
/var/folders/d2/lsvbp7p92gnc5ql59sc72rx40000gn/T/ipykernel_44548/2575044614.py:37: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if idx % 100 is 0:
/var/folders/d2/lsvbp7p92gnc5ql59sc72rx40000gn/T/ipykernel_44548/2575044614.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dropped_routes["dropped_q_over_q"] = True


YEAR: 1997
QUARTER: 1
IDX: 19800
IDX: 19900
IDX: 20000
IDX: 20100
IDX: 20200
IDX: 20300
IDX: 20400
IDX: 20500
IDX: 20600
IDX: 20700
IDX: 20800
IDX: 20900
IDX: 21000
IDX: 21100
IDX: 21200
IDX: 21300
IDX: 21400
IDX: 21500
IDX: 21600
IDX: 21700
IDX: 21800
IDX: 21900
IDX: 22000
IDX: 22100
IDX: 22200
IDX: 22300
IDX: 22400
IDX: 22500
IDX: 22600
IDX: 22700
IDX: 22800
IDX: 22900
IDX: 23000
IDX: 23100
IDX: 23200
IDX: 23300
IDX: 23400
IDX: 23500
IDX: 23600
IDX: 23700
IDX: 23800
IDX: 23900
IDX: 24000
IDX: 24100
IDX: 24200
IDX: 24300
IDX: 24400
IDX: 24500
IDX: 24600
QUARTER: 2


/var/folders/d2/lsvbp7p92gnc5ql59sc72rx40000gn/T/ipykernel_44548/2575044614.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dropped_routes["dropped_q_over_q"] = True


IDX: 24700
IDX: 24800
IDX: 24900
IDX: 25000
IDX: 25100
IDX: 25200
IDX: 25300
IDX: 25400
IDX: 25500
IDX: 25600
IDX: 25700
IDX: 25800
IDX: 25900
IDX: 26000
IDX: 26100
IDX: 26200
IDX: 26300
IDX: 26400
IDX: 26500
IDX: 26600
IDX: 26700
IDX: 26800
IDX: 26900
IDX: 27000
IDX: 27100
IDX: 27200
IDX: 27300
IDX: 27400
IDX: 27500


IDX: 27600


In [ ]:
print(len(total_cancelled))
total_cancelled

,Year,quarter,citymarketid_1,citymarketid_2,city1,city2,nsmiles,passengers,fare,carrier_lg,...,Geocoded_City1,Geocoded_City2,tbl6pk,state_1,state_2,citadel_id,pass_pct_chng,fare_pct_chng,F_pass_pct_chng,F_fare_pct_chng
14718,1996,4,34252,30852,"Pasco/Kennewick/Richland, WA","Washington, DC (Metropolitan Area)",2187,34,485.98,DL,...,"Pasco/Kennewick/Richland, WA\n(37.123874, -76....","Washington, DC (Metropolitan Area)\n(38.892062...",2.000000e+14,WA,DC,3085234252,NaN,NaN,False,False
14719,1996,4,30792,31650,"Buffalo, NY","Minneapolis/St. Paul, MN",734,66,244.43,NW,...,"Buffalo, NY\n(39.945422, -78.64671)","Minneapolis/St. Paul, MN\n(44.977479, -93.264346)",2.000000e+14,NY,MN,3079231650,NaN,NaN,False,False
14720,1996,4,30977,34236,"Chicago, IL","Paso Robles/San Luis Obispo, CA",1820,12,272.75,AA,...,"Chicago, IL\n(41.775002, -87.696388)","Paso Robles/San Luis Obispo, CA\n(35.62565, -1...",2.000000e+14,IL,CA,3097734236,NaN,NaN,False,False
14721,1996,4,32448,31454,"Jackson/Vicksburg, MS","Orlando, FL",587,45,137.52,DL,...,"Jackson/Vicksburg, MS\n(30.325968, -81.65676)","Orlando, FL\n(28.538331, -81.378879)",2.000000e+14,MS,FL,3145432448,NaN,NaN,False,False
14722,1996,4,31650,30559,"Minneapolis/St. Paul, MN","Seattle, WA",1399,632,196.84,NW,...,"Minneapolis/St. Paul, MN\n(44.977479, -93.264346)","Seattle, WA\n(47.603229, -122.33028)",2.000000e+14,MN,WA,3055931650,NaN,NaN,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19697,1996,4,31041,32457,"Carlsbad, CA","San Francisco, CA (Metropolitan Area)",422,11,85.13,OE,...,"Carlsbad, CA\n(32.77647, -79.931027)","San Francisco, CA (Metropolitan Area)\n(37.780...",2.000000e+14,CA,CA,3104132457,NaN,NaN,False,False
19698,1996,4,32575,33964,"Los Angeles, CA (Metropolitan Area)","North Bend/Coos Bay, OR",755,11,151.33,AS,...,"Los Angeles, CA (Metropolitan Area)\n(34.05223...","North Bend/Coos Bay, OR\n(42.130979, -87.831558)",2.000000e+14,CA,OR,3257533964,NaN,NaN,False,False
19699,1996,4,30666,32575,"Bellingham, WA","Los Angeles, CA (Metropolitan Area)",1071,53,134.62,AS,...,"Bellingham, WA\n(41.698264, -88.077915)","Los Angeles, CA (Metropolitan Area)\n(34.05223...",2.000000e+14,WA,CA,3066632575,NaN,NaN,False,False
19700,1996,4,32575,34433,"Los Angeles, CA (Metropolitan Area)","Santa Rosa, CA",433,41,104.78,OE,...,"Los Angeles, CA (Metropolitan Area)\n(34.05223...","Santa Rosa, CA\n(38.439698, -122.715642)",2.000000e+14,CA,CA,3257534433,NaN,NaN,False,False
